In [1]:
from edc import check_compatibility
check_compatibility("user-0.19.5")

This notebook is compatible with this base image version (user-0.19.5).

# First steps on the Euro Data Cube platform

Euro Data Cube provides a JupyterLab environment, which automatically provides **credentials** for services with **active subscriptions** as **environment variables.**

## Setup
The following platform setup function takes care of initializing your environment. It needs to be **called in every notebook**:

In [2]:
from edc import setup_environment_variables
setup_environment_variables()

API credentials have automatically been injected for your active subscriptions.  
The following environment variables are now available:
* `SH_CLIENT_ID`, `SH_CLIENT_NAME`, `SH_CLIENT_SECRET`, `SH_INSTANCE_ID`

The following additional environment variables have been loaded from `~/custom.env`:
* `AWS_BUCKET`
* `DAPA_URL`
* `DB_HOST`, `DB_NAME`, `DB_PASSWORD`, `DB_USER`
* `OGC_EDC_URL`
* `REFERENCE_DATA`


As you can see here, the credentials mentioned above are now part of your environment variables.  
You can also print them, but make sure to **keep them confidential**!


In [3]:
import os
'SH_CLIENT_SECRET' in os.environ

True

Usually, it is not even necessary to access the credentials.  
Many libraries such as `xcube_sh` or `xcube_geodb` load them directly from the environment by default.

## Retrieving data

Let's say you want to retrieve some Sentinel Data as defined by the following cube (don't worry if you don't understand all the details here, it will be explained <a target="_blank" href="https://eurodatacube.com/marketplace/notebooks/getting-started/EDC_Sentinel_Hub-XCUBE_integration.ipynb">here</a>):

In [4]:
from xcube_sh.config import CubeConfig
cube_config = CubeConfig(
    dataset_name="S2L2A",
    band_names=["B04", "B08"],
    tile_size=[512, 512],
    geometry=(10.00, 54.27, 10.30, 54.50),
    spatial_res=0.00018,
    time_range=["2018-05-02", "2018-05-26"],
    time_period="3D",
)

The following `xcube` call will fetch the data using your active SentinelHub subscription automatically using the credentials from the environment variables:

In [5]:
from xcube_sh.cube import open_cube
cube = open_cube(cube_config)
cube.B04.lat

<xarray.DataArray 'lat' (lat: 1536)>
array([54.54639, 54.54621, 54.54603, ..., 54.27045, 54.27027, 54.27009])
Coordinates:
  * lat      (lat) float64 54.55 54.55 54.55 54.55 ... 54.27 54.27 54.27 54.27
Attributes:
    units:          decimal_degrees
    long_name:      longitude
    standard_name:  latitude

## Explicit credential handling for advanced libraries

Some libraries such as the EO-Learn ML library do require you to set up credentials manually. Since the credentials are available using environment variables, there are different means of using them.

If you plan to use for instance the SentinelHub command line configuration tool (as recommended by EO-learn), you can make use of the environment variable expansion of the shell:

In [6]:
!sentinelhub.config --sh_client_id $SH_CLIENT_ID > /dev/null

See <a target="_blank" href="https://eurodatacube.com/marketplace/notebooks/contributions/EDC_Usecase-LPIS_Crop-Type-Classification.ipynb">this notebook</a> for a complete example.

If you need to pass the credentials in Python, the IPython magic command `%env` is a means of achieving that:

In [7]:
client_id = %env SH_CLIENT_ID

You can however also access the credentials using idiomatic python:

In [8]:
client_id = os.environ['SH_CLIENT_ID']

## Adding custom credentials e.g. for S3 buckets

Up until now, we have seen how to use credentials from services activated via Euro Data Cube. If you want to use external services, such as Amazon S3 buckets, you can add the relevant credentials and configuration to a custom environment variable file.

This will allow you to use them in all of your notebooks and also to share the notebooks without publishing sensitive information. Users who install your contributed notebook enter their respective credentials in their respective file.

To create the your custom environment variable file, just create a text file called `custom.env` in your JupyterLab home directory and add your custom content as shown in this example:
```
# file contents of "custom.env"
AWS_BUCKET=foo_bar
OTHER_EXTERNAL_SERVICE_CONFIG=3
```

Now this file will automatically be included in the `setup_environment_variables()` as shown by its output:
> ```python
> from edc import setup_environment_variables
> setup_environment_variables()
> ```

> API credentials have automatically been injected for your active subscriptions.
> The following environment variables are now available:
>
> *   `SH_INSTANCE_ID`
> *   `SH_CLIENT_ID`
> *   `SH_CLIENT_SECRET`
> *   `SH_CLIENT_NAME`
>
> The following additional environment variables have been loaded from ~/custom.env:
>
> *   `AWS_BUCKET`
> *   `OTHER_EXTERNAL_SERVICE_CONFIG`
 


## Additional reference data and resources

Some notebooks require reference data like geojson files
> *   `REFERENCE_DATA`

use resources like a POSTGIS database for data management 
> *   `DB_HOST`
> *   `DB_NAME`
> *   `DB_USER`
> *   `DB_PASSWORD`

or need AWS credentials for accessing S3 buckets
> *   `AWS_BUCKET`

Such requirements are indicated by the use of environment variables within the notebook code and require addition **manual** setup upfront. Relevant artifacts (e.g. database creation scripts) are provided here **<a target="_blank" href="https://github.com/eurodatacube/notebooks/tree/master/artifacts">here</a>**.

## Conclusion

The Euro Data Cube platform allows you to focus on your data without needing to think about authentication and login credentials.

You can even **share notebooks** in the Euro Data Cube marketplace without any changes required. If anyone else runs this notebook on this platform, their respective API service credentials will be used automatically.

To learn more about using **SentinelHub** or **GeoDB** on the Euro Data Cube platform, check out **<a target="_blank" href="https://eurodatacube.com/marketplace/notebooks?tag=Getting%20started">these notebooks</a>** in the marketplace.